# Analisis de RNA-seq
¡Requerido activar ambiente miniconda en servidor dedicado *RNAseq*! \
Explora los ambientes disponibles con **conda env list**  

En el *práctico 2* se utilizó el software **fastq_screen** como un primer control de calidad para mapear los reads en formato fastq frente a un conjunto de genomas de referencia y lograr identificar su origen

In [ ]:
# Ejecutar en cluster dedicado
fastq_screen ../../../data/rnaseq_MTWT/*.fastq --outdir ./

Luego se ejecutó **FASTQC** para generar reportes de calidad de los datos de secuenciación para poder indentificar posibles problemas de lectura y contenido. Importante en el preprocesamiento de los datos

In [ ]:
fastqc ../../data/rnaseq_MTWT/*.fastq -o ./

### **Pre-procesamiento de RNA-seq**
Prueba inicial para corte de adaptadores y filtrado de secuencias utilizando CutAdapt 5.1 con Python 3.10.18 siguiendo la siguiente línea de comandos.\
> **Nota:**\
> La lista de adaptadores y de posibles contaminantes fue obtenida de \
> https://support-docs.illumina.com/SHARE/AdapterSequences/Content/SHARE/AdapterSeq/Nextera/SequencesNextera_Illumina.htm

In [ ]:
# Ejecución de la linea de comandos sobre ambiente miniconda con los programas respectivos instalados

cutadapt -a CTGTCTCTTATACACATCT -A CTGTCTCTTATACACATCT --action trim -u -2 -U -2 \
-q 30 -Q 30 --poly-a --trim-n --pair-filter=any --minimum-length 50 \
-o WT1_R1_out.fastq -p WT1_R2_out.fastq ../../data/rnaseq_MTWT/WT1_Pax6_R1_subset.fastq ../../data/rnaseq_MTWT/WT1_Pax6_R2_subset.fastq > reportWT1.txt

Siempre ejecutar **FastQC** para verificar la calidad del filtrado y modificaciones de los reads

In [ ]:
fastqc *.fastq -o quality_ctrl/

### **Alineamiento de secuencias**

**¿Qué es un archivo GTF?**
El **GTF (Gene Transfer Format)** es un archivo de texto tabulado utilizado para almacenar **anotaciones genómicas** como genes, transcritos, exones, CDS y UTRs.  
Es una variante estandarizada de **GFF2**, diseñada específicamente para representar estructuras génicas y transcriptómicas.

---
#### **Estructura del archivo**

Cada línea (excepto comentarios que comienzan con `#`) contiene **9 columnas obligatorias**:

1. **seqname** → Cromosoma/contig (ej. `chr1`, `MT`).  
2. **source** → Fuente de la anotación (ej. `ENSEMBL`, `HAVANA`).  
3. **feature** → Tipo de elemento (`gene`, `transcript`, `exon`, `CDS`, etc.).  
4. **start** → Posición inicial (*1-based, inclusive*).  
5. **end** → Posición final (*1-based, inclusive*).  
6. **score** → Valor numérico o `.` si no aplica.  
7. **strand** → Hebra (`+`, `-`, o `.`).  
8. **frame** → Fase de codón (`0`, `1`, `2`, o `.`).  
9. **attributes** → Pares clave-valor con metadatos.
---
#### **Ejemplo**

```txt
chr19    HAVANA    exon    409006    409170    .    -    .    gene_id "ENSG00000183186.7"; transcript_id "ENST00000332235.7"; exon_number 1; gene_name "C2CD4C";


